In [43]:
%serialconnect

Connecting to Serial /dev/ttyUSB0 baud=115200 
I (5807) network: event 1
ets Jun  8 2016 00:22:57

rst:0x1 (POWERON_RESET),boot:0x17 (SPI_FAST_FLASH_BOOT)
configsip: 0, SPIWP:0xee
clk_drv:0x00,q_drv:0x00,d_drv:0x00,cs0_drv:0x00,hd_drv:0x00,wp_drv:0x00
mode:DIO, clock div:2
load:0x3fff0018,len:4
load:0x3fff001c,len:4332
load:0x40078000,len:0
load:0x40078000,len:10992
entry 0x4007a6c4
W (64) rtc_clk: Potentially bogus XTAL frequency: 35 MHz, guessing 40 MHz
LOG 1 ON 1223
Scanning wifis: DoESLiverpool, DoESDinky, DoESBackup, BV6000


*** Sending Ctrl-C

I (5805) network: event 1
Traceback (most recent call last):
  File "main.py", line 7, in <module>
  File "connectwifi.py", line 14, in <module>
  File "connectwifi.py", line 12, in <genexpr>
KeyboardInterrupt: 
MicroPython v1.9.2-443-g236297f4 on 2017-11-14; ESP32 module with ESP32
Type "help()" for more information.
>>> 
>>>[reboot detected 0]repl is in normal command mode
[\r\x03\x03] b'\r\n>>> '
[\r\x01] b'\r\n>>> \r\nraw REPL; CTRL-B 

In [14]:
%sendtofile --quiet --source ../basicsockets/syslog.py
%sendtofile --quiet --source ../essential_sensor_code/asyncdevices.py

%sendtofile --quiet --source ../wificodes.txt
%sendtofile --quiet --source ../basicsockets/connectwifi.py
%sendtofile --quiet --source ../basicsockets/connecthotspot.py

%sendtofile --quiet --source ../async_web_serve/webserve_funcs.py


Sent 34 lines (713 bytes) to syslog.py.
Sent 48 lines (1507 bytes) to asyncdevices.py.
Sent 4 lines (87 bytes) to wificodes.txt.
Sent 33 lines (995 bytes) to connectwifi.py.
Sent 9 lines (327 bytes) to connecthotspot.py.
Sent 190 lines (6050 bytes) to webserve_funcs.py.
Sent 86 lines (3110 bytes) to websocket_funcs.py.


In [46]:
%sendtofile --source uploadfiletest.html


Sent 13 lines (338 bytes) to uploadfiletest.html.


In [10]:
%comment This does the main file loading

%sendtofile --source plain_main.py main.py

# you need to reset properly and monitor on serial line through picocom, sa

Sent 53 lines (1478 bytes) to main.py.


In [ ]:
# Below are the lines you can use to build the webserver in immediate mode

In [11]:
%rebootdevice

repl is in normal command mode
[\r\x03\x03] b'\r\nMicroPython v1.9.2-443-g236297f4 on 2017-11-14; ESP32 module with ESP32\r\nType "help()" for more information.\r\n>>> \r\n>>> \r\nPYB: soft reboot\r\nTraceback (most recent call last):\r\n  File "main.py", line 2, in <module>\r\nKeyboardInterrupt: \r\nMicroPython v1.9.2-443-g236297f4 on 2017-11-14; ESP32 module with ESP32\r\nType "help()" for more information.\r\n>>> \r\n>>> '
[\r\x01] b'\r\n>>> \r\nraw REPL; CTRL-B to exit\r\n>'

In [47]:
%disconnect


Closing serial Serial<id=0x7fae9b3f5518, open=True>(port='/dev/ttyUSB0', baudrate=115200, bytesize=8, parity='N', stopbits=1, timeout=0.5, xonxoff=False, rtscts=False, dsrdtr=False)


In [5]:
#%%writefile plain_main.py

import time
time.sleep(1.0)

from syslog import log, elog, powid
import os, ustruct, uasyncio
uasyncio.set_debug(0)

from connectwifi import ipnumber
#from connecthotspot import ipnumber


from webserve_funcs import servestaticfile
from webserve_funcs import servefilelist
from webserve_funcs import readhttpheaders
from webserve_funcs import recpostsave
from webserve_funcs import servemessage
from webserve_funcs import convertsavepostfile



LOG 44 ON 0
W (53648) phy_init: failed to load RF calibration data (0x1102), falling back to full calibration
Scanning wifis: DoESLiverpool, DoESDinky, DoESBackup, BV6000
I (56258) network: event 1
LOG 47 Strongest wifi DoESLiverpool
> auth (b0)
> assoc (0)
> run (10)
I (58378) network: event 4
.
.I (65008) network: GOT_IP
LOG 56 Device has ipnumber 192.168.0.72


In [6]:
#%%writefile --append plain_main.py

async def handleconnection(reader, writer):
    receivedrequest = await readhttpheaders(reader)
    
    print(receivedrequest)
    if "POST" in receivedrequest:
        await recpostsave(receivedrequest, reader, "POSTUPLOAD.txt")
        await servemessage(writer, "done")
        await writer.aclose()
        convertsavepostfile("POSTUPLOAD.txt")
        return

    rpath = receivedrequest.get("GET")
    log(str(receivedrequest))
    
    bcallaclose = True
    if rpath == "":
        await servefilelist(writer)
    elif rpath:
        await servestaticfile(writer, rpath)
        
    if bcallaclose:
        await writer.aclose()



In [7]:
#%%writefile --append plain_main.py

# enables rewriting handleconnection function without restarting
def handleconnection_indirect(reader, writer):
    try:
        return handleconnection(reader, writer)
    except OSError as e:
        print("handleconnection exception", str(e))

port = 80
loop = uasyncio.get_event_loop()
print("* Running on http://%s:%s/" % (ipnumber, port))
loop.create_task(uasyncio.start_server(handleconnection_indirect, ipnumber, port))


* Running on http://192.168.0.72:80/


In [8]:
#%%writefile --append plain_main.py

loop.run_forever()


.{'GET': ''}
LOG 61 {'GET': ''} None
[] serving file list
{'GET': 'syslog.txt'}
LOG 64 {'GET': 'syslog.txt'} None
about to serve syslog.txt
File syslog.txt (124 bytes)-- served in 4ms
.{'GET': 'syslog.py'}
LOG 69 {'GET': 'syslog.py'} None
about to serve syslog.py
File syslog.py (713 bytes)-- served in 11ms
.{'GET': 'lib/uasyncio/__init__.py'}
LOG 77 {'GET': 'lib/uasyncio/__init__.py'} None
about to serve lib/uasyncio/__init__.py
File lib/uasyncio/__init__.py (7999 bytes)-- served in 314ms
................................................................................................................................................

*** Sending Ctrl-C



Traceback (most recent call last):
  File "<stdin>", line 3, in <module>
  File "/lib/uasyncio/core.py", line 77, in run_forever
  File "/lib/uasyncio/__init__.py", line 69, in wait
KeyboardInterrupt: 


In [244]:
%sendtofile --source plain_main.py main.py

Sent 56 lines (1533 bytes) to main.py.
